#NinaPro DB5 dataset(電極番号1~8)

ジェスチャ：6種類

準拠するパラメータ：

・batch size = 128

・L2正則化の重み = 0.001

・学習率減衰：1~19 = 0.01, 20~39 = 0.001, 40~50 = 0.0001

・データ比：train:valid:test = 6:1:3

In [1]:
import numpy as np
N = np.empty((2,3,2))
A = np.array([[[11, 12], [13, 14], [15, 16]], [[21, 22], [23, 24], [25, 26]]])
B = np.array([[[51, 52], [53, 54], [55, 56]], [[61, 62], [63, 64], [65, 66]]])
C = np.array([[[51, 52], [53, 54], [55, 56]], [[61, 62], [63, 64], [65, 66]]])
#D = np.concatenate(A[:, :, 1].reshape(2, 3, 1), B[:, :, 1].reshape(2, 3, 1), C[:, :, 0].reshape(2, 3, 1), axis = 2)
D = np.stack([A[:, :, 1], B[:, :, 1], C[:, :, 0]], axis = 2)
#D = np.stack([N, A], axis = 2)
#D = np.stack([D, B], axis = 2)

#print(D)


In [2]:
!nvidia-smi

Tue Dec 20 19:33:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.94       Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   31C    P8    21W / 350W |    556MiB / 24576MiB |     28%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

dataset
バッチサイズ→128
テストデータ割合→10%

データセットの解凍

In [3]:
#import keras
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import Dense, Dropout, Flatten
#from tensorflow.keras.layers.advanced_activations import PReLU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler #学習率減衰
from tensorflow.keras import regularizers #L2正則化
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import time
import tensorflow as tf
import os

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'#GPUのメモリ確保を制限

#乱数の設定
os.environ['PYTHONHASHSEED'] = '0'
tf.random.set_seed(1)
np.random.seed(1)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

各種設定(データセットの指定など)

In [4]:
train_data_path = 'datasetNina_ExB_1' #データセットのフォルダ名

channel = 8 
folder = list(map(str, range(1, 7))) #データセットのクラス名

class_number = len(folder)
print('今回のデータで分類するクラス数は「', str(class_number), '」です。')

今回のデータで分類するクラス数は「 6 」です。


データセットの読み込み・準備

In [5]:
start = time.time()
X_image = []
Y_label = []
Xappend = X_image.append
Yappend = Y_label.append

#クラスごとのループ
for index, name in enumerate(folder): #name:クラスフォルダ名, index:クラスに対応している変数
  read_data = train_data_path + '/' + name

  files = glob.glob(read_data + '/*.npy') #クラスごとのファイルすべてを格納
  #print('--- 読み込んだデータセットは', read_data, 'です。')

  #ファイルごとのループ
  for i, file in enumerate(files):  #i:クラス内の各ファイルに対応している変数
    print('\r', read_data, ': ', i+1, '/', len(files), end='')
    array = np.load(file)

    Xappend(array) #画像データをndarrayとして格納
    Yappend(index) #画像に対応したクラス(画像ラベル)を格納

  print(' --finish: ' + str(int((time.time()-start)/60)) + 'm' + str(int((time.time()-start))%60) + 's')
  
#データセット配列の設定
X_image = np.array(X_image) #データセット全体の画像行列データ＝入力データ
Y_label = np.array(Y_label) #データセット全体の画像ラベル

#画像行列データを正規化
print(X_image.max(), X_image.min())
scale = (X_image.max() - X_image.min())/2
med = (X_image.max() + X_image.min())/2
X_image = (X_image - med)/scale
print(X_image.max(), X_image.min())

Y_label = to_categorical(Y_label, class_number) #Kerasではクラスをベクトルで扱うらしい【おまじない】

#訓練データとテストデータを分割。
x_train, valid_images, y_train, valid_labels = train_test_split(X_image, Y_label, test_size=0.4, random_state=0)
x_valid, x_test, y_valid, y_test = train_test_split(valid_images, valid_labels, test_size=0.75, random_state=0)

print('train data size: ', len(x_train))
print('valid data size: ', len(x_valid))
print('test data size: ', len(x_test))

 datasetNina_ExB_1/1 :  9534 / 9534 --finish: 0m39s
 datasetNina_ExB_1/2 :  7892 / 7892 --finish: 1m13s
 datasetNina_ExB_1/3 :  7954 / 7954 --finish: 1m46s
 datasetNina_ExB_1/4 :  8105 / 8105 --finish: 2m19s
 datasetNina_ExB_1/5 :  7072 / 7072 --finish: 2m50s
 datasetNina_ExB_1/6 :  7231 / 7231 --finish: 3m23s
456.3779591234996 -168.4499504384696
1.0 -1.0
train data size:  28672
valid data size:  4779
test data size:  14337


ch数(1~7)を指定したCNNモデル

In [6]:
BATCH_SIZE = 128
image_size_x = 25 #画像サイズ
image_size_y = 15
epochnumber = 50
L2weight = 0.001

for channel in range(1, 8):

    #CNNアーキテクチャの設計
    model = Sequential()

    model.add(BatchNormalization(input_shape=(image_size_x, image_size_y, channel)))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3), padding='same'))

    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), padding='same'))

    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), padding='same'))

    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), padding='same'))

    model.add(GlobalAveragePooling2D())

    model.add(Dense(class_number, activation='softmax', kernel_regularizer=regularizers.l2(L2weight)))

    #model.summary()

    #学習率減衰の設定
    def step_decay(epoch):
        x = 0.01
        if epoch >= 20: x = 0.001
        if epoch >= 40: x = 0.0001
        return x
    lr_decay = LearningRateScheduler(step_decay)

    model.compile(loss='categorical_crossentropy', #損失関数の設定
                optimizer=Adam(learning_rate=0.01), #最適化アルゴリズムの設定
                metrics=['accuracy'] #評価関数の設定
                )

    #電極の選択
    from itertools import combinations
    num_ch = list(range(8))
    option_ch = list(combinations(num_ch, channel))
    savelist = np.empty([0, channel+2])

    #電極の選択方法(コンビネーション)でループ
    for ch in option_ch:
        print('ch number: ', ch)

        x_train1 = x_train[:, :, :, ch]
        x_valid1 = x_valid[:, :, :, ch]
        x_test1 = x_test[:, :, :, ch]

        start_time = time.time()

        #可視化
        history = model.fit(x_train1, y_train, 
                            batch_size=BATCH_SIZE, 
                            epochs=epochnumber, 
                            verbose=0, #学習過程を出力するかどうか
                            validation_data=(x_valid1, y_valid), 
                            callbacks=[lr_decay] #学習率減衰の設定
                            )
        
        #結果のプリント
        score = model.evaluate(x_test1, y_test, verbose=0)
        print('Computation time: {0:.3f} sec'.format(time.time() - start_time))
        print('Loss:', score[0], ' Accuracy:', score[1] * 100, '%') 

        model.save('model/EMGNet_Nina_B6_1_' + ''.join(map(str, ch)) + '.h5') #モデルの保存
        
        savedata = np.hstack((ch, score[0], score[1]*100)) #格納用の結果
        savelist = np.vstack((savelist, savedata)) #結果を保存用配列に格納

    #格納した結果のcsv保存
    savename = "EMGNet_Nina_B6_1_" + str(int(channel)) + "ch.csv"
    np.savetxt(savename, savelist, delimiter=",")


ch number:  (0,)
Computation time: 133.985 sec
Loss: 1.6994235515594482  Accuracy: 27.09771990776062 %
ch number:  (1,)
Computation time: 131.176 sec
Loss: 1.5851973295211792  Accuracy: 32.31498897075653 %
ch number:  (2,)
Computation time: 131.803 sec
Loss: 1.5724875926971436  Accuracy: 33.458882570266724 %
ch number:  (3,)
Computation time: 131.776 sec
Loss: 1.6319729089736938  Accuracy: 31.338495016098022 %
ch number:  (4,)
Computation time: 131.604 sec
Loss: 1.532806396484375  Accuracy: 37.288135290145874 %
ch number:  (5,)
Computation time: 126.880 sec
Loss: 1.6496639251708984  Accuracy: 31.031596660614014 %
ch number:  (6,)
Computation time: 124.153 sec
Loss: 1.6808748245239258  Accuracy: 28.694984316825867 %
ch number:  (7,)
Computation time: 123.464 sec
Loss: 1.5720829963684082  Accuracy: 32.28708803653717 %
ch number:  (0, 1)
Computation time: 125.671 sec
Loss: 1.3784548044204712  Accuracy: 44.10267174243927 %
ch number:  (0, 2)
Computation time: 125.104 sec
Loss: 1.3656717538